# 

In [35]:
import torch
import pandas as pd
import arviz as az
from scipy.optimize import curve_fit
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Spectral
from bokeh.models import Slope, Div
from bokeh.io import curdoc,output_notebook
from bokeh.layouts import column,gridplot
#from print_versions import print_versions
from sklearn.metrics import r2_score
from bokeh.models import Band, ColumnDataSource
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,examine,help"
output_notebook()
import numpy as np

Loading BokehJS ...

In [5]:
PM25=pd.read_excel("../data/pm_flow.xlsx")

In [6]:
train,test=PM25.loc[PM25.label==0,:],PM25.loc[PM25.label==1,:]

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
train.shape,test.shape

((926, 24), (194, 24))

In [9]:
gpr=torch.load("../models/PM2.5_flow",weights_only=False).to(device);

In [10]:
exp_curve=curve_fit(lambda t,a,b: a*np.exp(b*t),  train.flow,train.pm_final);
linear_curve=curve_fit(lambda t,a,b: a * t +b,  train.flow,train.pm_final);
log_curve=curve_fit(lambda t,a,b: a * t +b,  train.flow,np.log(train.pm_final));

In [11]:
def exp(x, a, b):
    return a * np.exp(b * x);
def linear(x, a, b):
    return a * x +b;


In [12]:
x=np.linspace(3.6,4.6,400)
x_tor=torch.tensor(x).float().to(device)
data=pd.DataFrame(x,index=np.arange(400),columns=["flow"])
data["lieanr_scale_pm"]=(linear(4,*linear_curve[0])/linear(x,*linear_curve[0]))
data["exp_scale_pm"]=exp(4,*exp_curve[0])/exp(x,*exp_curve[0])
data["guassion_scale_pm"]=(gpr.to(device)(torch.tensor(4*np.ones(len(x))).float().to(device))[0]/gpr.to(device)(x_tor)[0]).cpu().detach().numpy()
data["log_scale_pm"]=linear(4,*log_curve[0])/linear(x,*log_curve[0])

In [13]:
r2_score(test.pm_final,linear(test.flow,*linear_curve[0])),r2_score(test.pm_final,gpr.to(device)(torch.tensor(test.flow.values).float().to(device))[0].cpu().detach().numpy()),r2_score(test.pm_final,exp(test.flow,*exp_curve[0])),r2_score(test.pm_final,linear(test.flow,*linear_curve[0]))

(-0.8128754277658401,
 -0.6942414645883863,
 -0.8006949433370121,
 -0.8128754277658401)

In [14]:
def rmse(y_raw,y_predict):
    return np.sqrt(np.mean(((y_raw-y_predict)**2)))

In [15]:
#rmse(test.pm_final,linear(test.flow,*linear_curve[0])),rmse(test.pm_final,gpr(torch.tensor(test.flow.values).float())[0].detach().numpy()),rmse(test.pm_final,exp(test.flow,*exp_curve[0])),rmse(test.pm_final,linear(test.flow,*linear_curve[0]))

In [16]:
gpr_bc=torch.load("../models/bc_flow",weights_only=False,map_location='cuda');

In [17]:
exp_curve_bc=curve_fit(lambda t,a,b: a*np.exp(b*t),  train.flow,train.sv_final)
linear_curve_bc=curve_fit(lambda t,a,b: a * t +b,  train.flow,train.sv_final)
log_curve_bc=curve_fit(lambda t,a,b: a * t +b,  train.flow,np.log(train.sv_final))

In [18]:
X=torch.tensor(train.flow.values).float().to(device)
y=torch.tensor(train.sv_final.values).float().to(device)
y_pm=torch.tensor(train.pm_final.values).float().to(device)

In [19]:
gpr

GPRegression(
  (kernel): RBF()
)

In [20]:
train.loc[:,"PM25_Gaussion"]=(y_pm*gpr(torch.tensor(4*np.ones(len(X))).float().to(device))[0]/gpr(X)[0]).cpu().detach().numpy()
train.loc[:,"pm25_linear"]=linear(4,*linear_curve[0])/linear(train.flow.values,*linear_curve[0])*train.pm_final.values
train.loc[:,"pm25_exp"]=exp(4,*exp_curve[0])/exp(train.flow,*exp_curve[0])*train.pm_final.values
train.loc[:,"pm25_log"]=linear(4,*log_curve[0])/linear(train.flow,*log_curve[0])*train.pm_final.values

/tmp/ipykernel_1210841/241013154.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.loc[:,"PM25_Gaussion"]=(y_pm*gpr(torch.tensor(4*np.ones(len(X))).float().to(device))[0]/gpr(X)[0]).cpu().detach().numpy()
/tmp/ipykernel_1210841/241013154.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.loc[:,"pm25_linear"]=linear(4,*linear_curve[0])/linear(train.flow.values,*linear_curve[0])*train.pm_final.values
/tmp/ipykernel_1210841/241013154.py:3: SettingWithCopyWarning: 
A value is trying to be set on 

In [21]:
train.loc[:,"bc_Gaussion"]=(y*gpr_bc(torch.tensor(4*np.ones(len(X))).float().to(device))[0]/gpr_bc(X)[0]).cpu().detach().numpy()
train.loc[:,"bc_linear"]=linear(4,*linear_curve_bc[0])/linear(train.flow,*linear_curve_bc[0])*train.sv_final
train.loc[:,"bc_exp"]=exp(4,*exp_curve_bc[0])/exp(train.flow,*exp_curve_bc[0])*train.sv_final
train.loc[:,"bc_log"]=linear(4,*log_curve_bc[0])/linear(train.flow,*log_curve_bc[0])*train.sv_final

/tmp/ipykernel_1210841/247581170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.loc[:,"bc_Gaussion"]=(y*gpr_bc(torch.tensor(4*np.ones(len(X))).float().to(device))[0]/gpr_bc(X)[0]).cpu().detach().numpy()
/tmp/ipykernel_1210841/247581170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.loc[:,"bc_linear"]=linear(4,*linear_curve_bc[0])/linear(train.flow,*linear_curve_bc[0])*train.sv_final
/tmp/ipykernel_1210841/247581170.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [22]:
X=torch.tensor(PM25.flow.values).float().to(device)
y=torch.tensor(PM25.sv_final.values).float().to(device)
y_pm=torch.tensor(PM25.pm_final.values).float().to(device)

In [23]:
PM25["BC_Gaussion"]=(y *gpr_bc(torch.tensor(4*np.ones(len(PM25.flow))).float().to(device))[0]/gpr_bc(X)[0]).cpu().detach().numpy()
PM25["BC_linear"]=linear(4,*linear_curve_bc[0])/linear(PM25.flow,*linear_curve_bc[0])*PM25.sv_final
PM25["BC_exp"]=exp(4,*exp_curve_bc[0])/exp(PM25.flow,*exp_curve_bc[0])*PM25.sv_final
PM25["BC_log"]=linear(4,*log_curve_bc[0])/linear(PM25.flow,*log_curve_bc[0])*PM25.sv_final

In [24]:
PM25["pm25_Gaussion"]=(y_pm *gpr(torch.tensor(4*np.ones(len(PM25.flow))).float().to(device))[0]/gpr(X)[0]).cpu().detach().numpy()
PM25["pm25_linear"]=linear(4,*linear_curve[0])/linear(PM25.flow,*linear_curve[0])*PM25.pm_final
PM25["pm25_exp"]=exp(4,*exp_curve[0])/exp(PM25.flow,*exp_curve[0])*PM25.pm_final
PM25["pm25_log"]=linear(4,*log_curve[0])/linear(PM25.flow,*log_curve[0])*PM25.pm_final

In [25]:
#x=np.linspace(3.4,4.6,400)
p1_bc=curve_fit(lambda t,a,b: a*np.exp(b*t),  PM25.flow,PM25.sv_final)
p2_bc=curve_fit(lambda t,a,b: a*np.exp(b*t),  PM25.flow,PM25.BC_linear)
p3_bc=curve_fit(lambda t,a,b: a * t +b,  PM25.flow,PM25.BC_exp)
p4_bc=curve_fit(lambda t,a,b: a * t +b,  PM25.flow,PM25.BC_Gaussion)

In [26]:
#x=np.linspace(3.4,4.6,400)
p1=curve_fit(lambda t,a,b: a*np.exp(b*t),  PM25.flow,PM25.pm_final)
p2=curve_fit(lambda t,a,b: a*np.exp(b*t),  PM25.flow,PM25.pm25_linear)
p3=curve_fit(lambda t,a,b: a * t +b,  PM25.flow,PM25.pm25_exp)
p4=curve_fit(lambda t,a,b: a * t +b,  PM25.flow,PM25.pm25_Gaussion)

In [27]:

p = figure(tools=TOOLS, toolbar_location=None)
p.scatter(PM25.flow, y=PM25.pm_final, color="blue", marker="dot", size=20, alpha=0.4);
#p.line(x, linear(x,*p1[0]), line_width=3,legend_label="Raw fit", line_color="red");
p.line(x, linear(x,*p2[0]), line_width=3,legend_label="Linaer correction", line_color="green");
p.line(x, linear(x,*p3[0]), line_width=3,legend_label="exponential", line_color="yellow");
p.line(x, linear(x,*p4[0]), line_width=3,legend_label="Gaussion correction", line_color="brown");
p1 = figure(tools=TOOLS, toolbar_location=None)
p1.scatter(PM25.flow, y=PM25.sv_final, color="blue", marker="dot", size=20, alpha=0.4);
#p.line(x, linear(x,*p1[0]), line_width=3,legend_label="Raw fit", line_color="red");
p1.line(x, linear(x,*p2_bc[0]), line_width=3,legend_label="Linaer correction", line_color="green");
p1.line(x, linear(x,*p3_bc[0]), line_width=3,legend_label="exponential", line_color="yellow");
p1.line(x, linear(x,*p4_bc[0]), line_width=3,legend_label="Gaussion correction", line_color="brown");

grid =  gridplot([[p, p1]], width=500, height=500);

show(column(grid));

In [28]:
r2_score(test.sv_final,linear(test.flow,*linear_curve_bc[0])),r2_score(test.sv_final.values,gpr_bc(torch.tensor(test.flow.values).float().to(device))[0].cpu().detach().numpy()),r2_score(test.sv_final,exp(test.flow,*exp_curve_bc[0])),r2_score(test.sv_final,linear(test.flow,*linear_curve_bc[0]))

(-0.06865764479939807,
 -0.24091083223104404,
 -0.06206324072081859,
 -0.06865764479939807)

In [29]:
rmse(test.sv_final,linear(test.flow,*linear_curve_bc[0])),rmse(test.sv_final,gpr_bc(torch.tensor(test.flow.values).float().to(device))[0].cpu().detach().numpy()),rmse(test.sv_final,exp(test.flow,*exp_curve_bc[0])),rmse(test.sv_final,linear(test.flow,*linear_curve_bc[0]))

(0.4343784751525119,
 0.4680791385227139,
 0.43303618379441206,
 0.4343784751525119)

In [30]:

data["lieanr_scale_pm"]=(linear(4,*linear_curve[0])/linear(x,*linear_curve[0]))
data["exp_scale_pm"]=exp(4,*exp_curve[0])/exp(x,*exp_curve[0])
data["guassion_scale_pm"]=(gpr(torch.tensor(4*np.ones(x_tor.shape)).float().to(device))[0]/gpr(x_tor)[0]).cpu().detach().numpy()
data["log_scale_pm"]=linear(4,*log_curve[0])/linear(x,*log_curve[0])

In [31]:

data["lieanr_scale_bc"]=(linear(4,*linear_curve_bc[0])/linear(x,*linear_curve_bc[0]))
data["exp_scale_bc"]=exp(4,*exp_curve_bc[0])/exp(x,*exp_curve_bc[0])
data["guassion_scale_bc"]=(gpr_bc(torch.tensor(4*np.ones(len(x_tor))).float().to(device))[0]/gpr_bc(x_tor)[0]).cpu().detach().numpy()
data["log_scale_bc"]=linear(4,*log_curve_bc[0])/linear(x,*log_curve_bc[0])

In [32]:
p = figure(title = "Penguin size", background_fill_color="#fafafa",tools=TOOLS,x_range=(0.6,2.5),y_range=(0.6,2.5))

p.line(data.guassion_scale_pm, data.guassion_scale_bc, line_color="blue",
        line_width=3, legend_label="Linear")
p.xaxis.axis_label = r'$$Scale\_PM2.5$$'
p.yaxis.axis_label = r'$$Scale\_black\_carbon$$'
p.xaxis.axis_label_text_font_size = "14px"
p.yaxis.axis_label_text_font_size = "16px"
p.yaxis.axis_label_text_font_style = "bold"
p.xaxis.axis_label_text_font_style = "bold"
show(p)

In [33]:
p = figure(title = r"$$PM_{2.5}$$", background_fill_color="#fafafa",tools=TOOLS,x_range=(3.6,4.6),y_range=(0.5,2))

p.line(data.flow, data.lieanr_scale_pm, line_color="blue", line_width=3, legend_label="Linear")
p.line(data.flow, data.exp_scale_pm, line_color="red", line_width=3, legend_label="Exponential")
p.line(data.flow, data.guassion_scale_pm, line_color="green",line_width=3, legend_label="Gaussian")
p.line(data.flow, data.log_scale_pm, line_color="yellow", line_width=3, legend_label="Log")
p.legend.location = "top_left"
p.legend.title = r"Correction"
p.legend.title_text_font_size = "16px"
p.legend.title_text_font_style = "bold"
p.xaxis.axis_label = r'$$Flow \frac{L}{s}$$'
p.yaxis.axis_label = 'scale'
p.xaxis.axis_label_text_font_size = "14px"
p.yaxis.axis_label_text_font_size = "16px"
p.yaxis.axis_label_text_font_style = "bold"
p.xaxis.axis_label_text_font_style = "bold"

p1 = figure(tools=TOOLS,title = "Black Carbon", background_fill_color="#fafafa",x_range=(3.6,4.6),y_range=(0.5,2))

p1.line(data.flow, data.lieanr_scale_bc, line_color="blue",
        line_width=3, legend_label="Linear")
p1.line(data.flow, data.exp_scale_bc, line_color="red", line_width=3, legend_label="Exponential")
p1.line(data.flow, data.guassion_scale_bc, line_color="green", line_width=3, legend_label="Gaussian")
p1.line(data.flow, data.log_scale_bc, line_color="yellow", line_width=3, legend_label="Log")
p1.legend.location = "top_left"
p1.legend.title = "Correction"
p1.legend.title_text_font_size = "16px"
p1.xaxis.axis_label_text_font_size = "14px"
p1.yaxis.axis_label_text_font_size = "16px"
p1.yaxis.axis_label_text_font_style = "bold"
p1.xaxis.axis_label_text_font_style = "bold"

p1.legend.title_text_font_style = "bold"
p1.xaxis.axis_label = r'$$Flow \frac{L}{s}$$'
p1.yaxis.axis_label = 'scale'
grid =  gridplot([[p, p1]], width=500, height=500)

show(column(grid))

In [34]:
PM25.to_excel("../data/pm25_BC_corrected.xlsx")